# CanDI and DESeq2
Let's say I want to look at changes in RNA expression across some cell lines in CCLE. DESeq2 is my preffered tool for doing differential expression analysis, unforutantely it's written in R. CanDI makes it easy to format CCLE read counts data into the shape that DESeq2 expects.

In [1]:
import CanDI.candi as can
import numpy as np
import pandas as pd

#### Object Instantiation
For this example I'm going to do differential expression analysis across male and female KRAS mutant cell lines. The cell below uses CanDI to generate the correct CellLineCluster objects for our purpose.

In [2]:
lung = can.Cancer("Lung Cancer", subtype = "NSCLC")
lung = can.CellLineCluster(lung.mutated("KRAS", variant = "Variant_Classification", item = "Missense_Mutation"))

lung_male = can.CellLineCluster(list(lung._info.loc[lung._info.sex == "Male",].index))
lung_female = can.CellLineCluster(list(lung._info.loc[lung._info.sex == "Female"].index))

mutations has not been loaded. Do you want to load, y/n?> y
Load Complete


#### Data Munging
The follow function takes two objects that we want to compare and automatically generates the counts and coldata matrices that DESeq2 needs to run. It's typically a good idea to filter our genes/transcripts with consistently low counts prior to running DESeq2. This speeds up analysis and avoids issues related to read count scaling and multiple hypthothesis testing corrections. In this case we don't care about different splicing of the same genes so I sum counts for duplicate indeces for all samples. 

In [3]:
def make_counts_coldata(obj1, obj2, condition, factor1, factor2):
    
    counts1 = obj1.rnaseq_reads
    coldat1 = pd.Series(counts1.shape[1] * [factor1], index = counts1.columns, name = condition)
    
    counts2 = obj2.rnaseq_reads
    coldat2 = pd.Series(counts2.shape[1] * [factor2], index = counts2.columns, name = condition)
    
    #Concatenate Column Data
    coldat = pd.concat([coldat1, coldat2], axis = 0)
    #Concatenate read count data 
    counts_mat = pd.concat([counts1, counts2], axis = 1)
    #Sum duplicate indeces
    counts_mat = counts_mat.groupby(counts_mat.index).sum().astype(int)
    
    return counts_mat, coldat
    
counts, coldat = make_counts_coldata(lung_male, lung_female, "sex", "male", "female")

counts.to_csv("temp_dat/lung_sex_counts.csv")
coldat.to_csv("temp_dat/lung_sex_coldata.csv")

rnaseq_reads has not been loaded. Do you want to load, y/n?> y
Load Complete


#### Running DESeq2
In the following cell I use the csvs I just saved as arguments for an r-script that runs DESeq2. The last argument in this script the filname for the results.

In [ ]:
!Rscript scripts/run_deseq.r temp_dat/lung_sex_counts.csv temp_dat/lung_sex_coldata.csv temp_dat/lung_sex_deseq.csv

#### Analyzing Results
Now we can read the results of the differential expression analysis back into our python enviroment and continue analysis as necessary. Looking at the genes with the lowest adjusted p-value we see that XIST is the most significant differentially expressed genes. XIST is a lncRNA responsible for X inactivation in females so this a good positive control for our analysis.

In [4]:
res = pd.read_csv("temp_dat/lung_sex_deseq.csv", index_col = "Unnamed: 0")
res.sort_values("padj").head()

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
XIST,3936.090666,-7.030433,0.708612,-9.921418,3.359498e-23,9.148248e-19
BCL2L15,435.882075,-5.505807,0.604359,-9.110166,8.225616e-20,7.466391e-16
FAM224B,10.235047,21.650886,2.367773,9.143987,6.019109e-20,7.466391e-16
CEACAM5,12273.358936,-7.444559,0.859163,-8.664898,4.519171e-18,3.076539e-14
GJB1,90.468162,-6.193651,0.741574,-8.352040,6.709420e-17,3.654085e-13
